<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Convert NLCD to COG

_by Joe Hamman (CarbonPlan), June 29, 2020_

This notebook converts local copies of NLDC rasters to COG

**Inputs:**

- local copies of NLCD data

**Outputs:**

- COGs published to cloud storage

**Notes:**

- No reprojection or processing of the data is done in this notebook.
- Includes both conus (L48) and alaska (AK)
- Paths here need to be harmonized as it was run in two different local
  environments


In [ ]:
import io
import os
import pathlib

from google.cloud import storage
from rasterio.io import MemoryFile
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles

storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB
storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB

In [ ]:
# This is the COG profile:
dst_profile = cog_profiles.get("deflate")
os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"
] = "/Users/freeman/.config/gcloud/legacy_credentials/jeremy@carbonplan.org/adc.json"


def upload(src, target, bucket="carbonplan-data"):

    storage_client = storage.Client("carbonplan")
    bucket = storage_client.bucket(bucket)
    blob = bucket.blob(target)
    blob.upload_from_filename(src)

## NLCD_Land_Cover_Change_Index_L48_20190424


In [ ]:
# raw file
source = "/Users/jhamman/workdir/carbonplan_data_downloads/mlrc/NLCD_Land_Cover_Change_Index_L48_20190424/NLCD_Land_Cover_Change_Index_L48_20190424.img"

# local target
target = "./raster.tif"

# This is where we'll write the COGs when we're done
cloud_target = (
    "raw/mlrc/NLCD_Land_Cover_Change_Index_L48_20190424/30m/raster.tif"
)

In [ ]:
# translate to COG
cog_translate(source, target, dst_profile)

# Upload to GCS
upload(target, cloud_target)

# Remove temporary file
os.remove(target)

## NLCD_Land_Cover_L48_20190424_full_zip


In [ ]:
target = "./raster.tif"

for year in [2001, 2004, 2006, 2008, 2011, 2013, 2016]:
    print(year)
    source = f"/Users/jhamman/workdir/carbonplan_data_downloads/mlrc/NLCD_Land_Cover_L48_20190424_full_zip/NLCD_{year}_Land_Cover_L48_20190424.img"
    cloud_target = f"raw/mlrc/NLCD_Land_Cover_L48_20190424_full/30m/{year}.tif"

    # translate to COG
    cog_translate(source, target, dst_profile)

    # Upload to GCS
    upload(target, cloud_target)

    # Remove temporary file
    os.remove(target)

## NLCD_Land_Cover_AK_20200213


In [ ]:
workdir = pathlib.Path("/Users/freeman/workdir/carbonplan_data_downloads/nlcd/")

target = "./raster.tif"

for year in [2016]:
    print(year)
    source = (
        workdir
        / f"NLCD_{year}_Land_Cover_AK_20200213/NLCD_{year}_Land_Cover_AK_20200213.img"
    )
    cloud_target = f"raw/nlcd/NLCD_Land_Cover_AK_20200213/30m/{year}.tif"

    # translate to COG
    cog_translate(source, target, dst_profile)

    # Upload to GCS
    upload(target, cloud_target)

    # Remove temporary file
    os.remove(target)